# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv("weather_data.csv")
#weather_df.head()
clean_weather_data=weather_df.dropna()
#clean_weather_data.head()
del clean_weather_data["Unnamed: 0"]
clean_weather_data.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,Soyo,100,AO,1666381795,86,-6.1349,12.3689,77.02,8.86
1,Riyadh,0,SA,1666381576,14,24.6877,46.7219,75.34,9.53
2,Hithadhoo,100,MV,1666381797,78,-0.6000,73.0833,80.82,10.38
3,Atuona,42,PF,1666381799,63,-9.8000,-139.0333,77.09,16.84
4,Longyearbyen,20,SJ,1666381800,63,78.2186,15.6401,24.64,17.27
5,Seredka,100,RU,1666381802,86,58.1584,28.1858,40.75,3.91
6,Yellowknife,75,CA,1666381803,74,62.4560,-114.3525,30.72,8.05
7,Tasiilaq,100,GL,1666381805,96,65.6145,-37.6368,32.20,3.51
8,Woodward,0,US,1666381806,16,36.4337,-99.3904,90.97,14.97
9,Saskylakh,100,RU,1666381808,95,71.9167,114.0833,11.95,5.03


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [6]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = clean_weather_data[['Lat','Lng']]
humidity = clean_weather_data['Humidity'].astype(float)

In [10]:
#plot heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,humidity,dissipating=False,max_intensity=100,point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Filter vacation with zero cloudiness
vacation_no_cloud = clean_weather_data[clean_weather_data["Cloudiness"] == 0]

# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temperature"] > 70]

# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temperature"] < 80]


# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed"] < 10]

# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity"] < 60]

# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,Riyadh,0,SA,1666381576,14,24.6877,46.7219,75.34,9.53
1,Palm Coast,0,US,1666381717,47,29.5850,-81.2078,73.99,9.22
2,Buraidah,0,SA,1666382048,13,26.3260,43.9750,75.65,8.39
3,Bilma,0,NE,1666382098,20,18.6853,12.9164,77.70,9.66
4,Mount Isa,0,AU,1666382024,32,-20.7333,139.5000,74.30,8.23
5,Lincoln,0,US,1666381958,22,40.8000,-96.6670,79.02,4.61
6,Saint George,0,US,1666381981,23,37.1041,-113.5841,77.43,3.00
7,Mopipi,0,BW,1666382271,24,-21.1833,24.8833,78.62,9.40
8,Jaisalmer,0,IN,1666382458,25,26.9147,70.9181,79.39,8.23
9,Vallenar,0,CL,1666382472,25,-28.5708,-70.7581,79.93,8.55


In [12]:
vaca_locations = indexed_perfect_vacation[["Lat", "Lng"]]

vaca_humidity = indexed_perfect_vacation["Humidity"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Lat"]
    lng = indexed_perfect_vacation.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Hotel Name"] = hotels
indexed_perfect_vacation

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Nearest Hotel,Hotel Name
0,Riyadh,0,SA,1666381576,14,24.6877,46.7219,75.34,9.53,"InterContinental Riyadh, an IHG Hotel","InterContinental Riyadh, an IHG Hotel"
1,Palm Coast,0,US,1666381717,47,29.5850,-81.2078,73.99,9.22,Hammock Beach Golf Resort & Spa,Hammock Beach Golf Resort & Spa
2,Buraidah,0,SA,1666382048,13,26.3260,43.9750,75.65,8.39,Mövenpick Qassim,Mövenpick Qassim
3,Bilma,0,NE,1666382098,20,18.6853,12.9164,77.70,9.66,Expeditions Ténére voyages,Expeditions Ténére voyages
4,Mount Isa,0,AU,1666382024,32,-20.7333,139.5000,74.30,8.23,ibis Styles Mt Isa Verona,ibis Styles Mt Isa Verona
5,Lincoln,0,US,1666381958,22,40.8000,-96.6670,79.02,4.61,Courtyard by Marriott Lincoln Downtown/Haymarket,Courtyard by Marriott Lincoln Downtown/Haymarket
6,Saint George,0,US,1666381981,23,37.1041,-113.5841,77.43,3.00,Best Western Plus Abbey Inn,Best Western Plus Abbey Inn
7,Mopipi,0,BW,1666382271,24,-21.1833,24.8833,78.62,9.40,Nearest hotel not found,Nearest hotel not found
8,Jaisalmer,0,IN,1666382458,25,26.9147,70.9181,79.39,8.23,Hotel Fifu,Hotel Fifu
9,Vallenar,0,CL,1666382472,25,-28.5708,-70.7581,79.93,8.55,Humacao Bed & Breakfast,Humacao Bed & Breakfast


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Lat", "Lng"]]


In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Hotel Name: {hotel}" for hotel in hotels])
                                                            
                                                           
vaca_fig.add_layer(markers)

# Display Map
vaca_fig



Figure(layout=FigureLayout(height='420px'))